In [ ]:
# The first step to obtain the data, is to request the API client and secret from Idealista. 
# It can be done here: https://developers.idealista.com/access-request

In [ ]:
# Once we have our credentials, we can start creating a function to generate our personalised token, which will 
# give us authorisation to extract data.

In [21]:
# Import the necessary libraries

import base64
import requests as rq
import json

In [23]:
# Define a function in order to obtain our personalised token

def get_oauth_token():
    '''
    This function will return our personalised token
    '''
    api_key = 'your api key'   # Your API key provided by Idealista
    secret = 'your secret'   # Your secred code provided by Idealista

    message = api_key + ":" + secret   # Combine the API key and the secret to get our personalised message

    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")   # Encode the message

    headers_dic = {"Authorization" : auth,
                   "Content-Type" : "application/x-www-form-urlencoded;charset=UTF-8"}   # Define our headers

    params_dic = {"grant_type" : "client_credentials",   # Define the request params
                  "scope" : "read"}

    r = rq.post("https://api.idealista.com/oauth/token",   # Perform the request with the api url, headers and params
                      headers = headers_dic,
                      params = params_dic)

    token = json.loads(r.text)['access_token']   # Obtain the personalised token, as a json

    return token

In [29]:
# The next step will be to define the search url. We will do this by defining the filter params of the search, 
# and combining them into the resulting url.

In [20]:
# This are the params we will use to filter our search

base_url = 'https://api.idealista.com/3.5/'     # Base search url
country = 'es'     # Search country (es, it, pt)                          
language = 'es'     # Search language (es, it, pt, en, ca) 
max_items = '50'     # Max items per call, the maximum set by Idealista is 50
operation = 'rent'     # Kind of operation (sale, rent) 
property_type = 'homes'     # Type of property (homes, offices, premises, garages, bedrooms)
order = 'priceDown'     # Order of the listings, consult documentation for all the available orders 
center = '40.4167,-3.70325'     # Coordinates of the search center
distance = '60000'     # Max distance from the center
sort = 'desc'     # How to sort the found items
bankOffer = 'false'     # If the owner is a bank
maxprice = '750'     # Max price of the listings

In [ ]:
# Define a function to obtain our search url

def define_search_url():
    '''
    This function will combine our params with the url, in order to create our own search url
    '''
    url = (base_url +      
           country +
           '/search?operation=' + operation +
           '&maxItems=' + max_items +
           '&order=' + order +
           '&center=' + center +
           '&distance=' + distance +
           '&propertyType=' + property_type +
           '&sort=' + sort + 
           '&numPage=%s' +
           '&maxPrice=' + maxprice +
           '&language=' + language)
    
    return url

In [ ]:
url = define_search_url()

In [30]:
# Now we need to create a function in order to do our search, with the data we already have.

In [ ]:
def search_api(url):  
    '''
    This function will use the token and url created previously, and return our search results.
    '''
    token = get_oauth_token()   #  Get the personalised token

    headers = {'Content-Type': 'Content-Type: multipart/form-data;',   # Define the search headers 
               'Authorization' : 'Bearer ' + token}

    content = rq.post(url, headers = headers)   # Return the content from the request

    result = json.loads(content.text)   # Transform the result as a json file

    return result

In [ ]:
# Since we need to give pagination to our search and this is our first search, we will set the pagination as 1
pagination = 1
first_search_url = url %(pagination)

In [ ]:
# Proceed to do the search with the paginated url
results = search_api(first_search_url)

In [28]:
# We have already gotten all the results, so we can start working in them.

In [ ]:
# First of all, we can extract 50 results/page, but there are more pages, so we have to define how many pages there are.

total_pages = results['totalPages']

In [25]:
# We can also save our first dataset. We will save it as an individual dataframe, but then we have to add it to a 
# bigger dataset, which will contains all our individual dataframes.

# We will create two functions for that.

In [ ]:
# Import the necessary libraries

import pandas as pd

In [ ]:
def results_to_df(results):
    '''
    This function will save the json results as a dataframe and return the resulting dataframe
    '''
    df = pd.DataFrame.from_dict(results['elementList'])

    return df

In [ ]:
def concat_df(df, df_tot):
    '''
    This function will take the main dataframe (df_tot), and concat it with the given individual dataframe, 
    returning the main dataframe 
    '''
    pd.concat([df_tot,df])
    
    return df_tot

In [ ]:
# Proceed to save the obtained results as a dataframe
df = results_to_df(results)

In [ ]:
# Since we still don't have a main dataframe where we can store all the data, we will create an empty dataframe
df_tot = pd.DataFrame()
df_tot = concat_df(df, df_tot)

In [ ]:
# At this point, we have all the necessary code to extract the data, but we need to extract data in
# all the pages of the results. For this, we will create a loop, which will perform a search in every of the pages

In [ ]:
# This will loop over all the pages in the search results. It start from 2, because we've already gotten the first page
for i in range(2, total_pages): 

    url = url %(i)   # Add the pagination to the url
    results = search_api(url)   # Get the search results
    df = results_to_df(results)   # Save the results as a dataframe
    df_tot = concat_df(df, df_tot)   # Concat the results to the main dataframe

In [31]:
# Once we have all our data, we just need to save it as a csv file, we have created the following function for that:

file_path = 'idealista.csv'

def df_to_csv(df):
    '''
    This function will take a given dataframe and save it as a csv file
    '''
    df = df.reset_index()   # Reset the index in order to organise the records
    df.to_csv(file_path, index=False)   # Save it into a csv

In [ ]:
# Run the function and you'll obtain a csv file with all the extracted data
df_to_csv(df_tot)

In [ ]:
# Idealista API has a limitation of 100 search queries/month, so this should be taken into 
# consideration when performing the search queries